In [1]:
import yfinance as yf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import pandas as pd
# import pandas_ta as ta
# import japanize_matplotlib
%matplotlib inline

/Users/otaryuki/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# ティッカーシンボル指定
ticker = "^N225"
# ticker = "^TOPX"
# ticker = "7203.T" # トヨタ
# ticker = "6758.T" # SONY
# ticker = "9434.T" # ソフトバンク
# ticker = "6502.T" # 東芝
# ticker = "8306.T" # 三菱UFJ
# ticker = "6501.T" # 日立
# ticker = "6861.T" # キーエンス
# ticker = "6098.T" # リクルート
# ticker = "9983.T" # ファストリ
# ticker = "9432.T" # NTT
stock = yf.Ticker(ticker)
start = "2009-01-01"
end = "2023-12-31"
data_master = stock.history(start = start, end = end)

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier

# windowsize=16
window_size = 16

data = data_master.copy()
data = data.dropna() # 念のため欠損を落とす
data["Return"] = data["Close"].pct_change() 
data = data.dropna()

all_predictions = []
all_actual = []
dates = data.index
length = len(data)

In [5]:
data.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Return
Date,,,,,,,,
2009-01-06 00:00:00+09:00,9130.009766,9171.030273,9029.940430,9080.839844,154800000,0.0,0.0,0.004171
2009-01-07 00:00:00+09:00,9133.799805,9325.349609,9106.049805,9239.240234,205600000,0.0,0.0,0.017443
2009-01-08 00:00:00+09:00,9143.209961,9148.830078,8876.419922,8876.419922,149900000,0.0,0.0,-0.039269
2009-01-09 00:00:00+09:00,8932.709961,8956.849609,8773.200195,8836.799805,137000000,0.0,0.0,-0.004464
2009-01-13 00:00:00+09:00,8732.629883,8732.929688,8405.500000,8413.910156,135600000,0.0,0.0,-0.047856


# ローリング法、特徴量：日次変化率

In [6]:
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import time

start_time = time.time()

start_i = window_size - 1 + 16
all_predictions = []
all_actual = []

# tqdmで進行状況を表示
for i in tqdm(range(start_i, length - 16), desc="Predicting"):
    # 特徴量
    X_current = data["Return"].iloc[i-window_size+1 : i+1].values.reshape(1, -1)
    
    # ラベル
    future_close = data["Close"].iloc[i+16]
    next_open = data["Open"].iloc[i+1]
    y_current = 1 if future_close > next_open else 0
    
    # 訓練データ作成
    train_X = []
    train_y = []
    for j in range(window_size-1, i):
        if j+16 < length:
            X_feat = data["Return"].iloc[j-window_size+1 : j+1].values
            future_c = data["Close"].iloc[j+16]
            next_o = data["Open"].iloc[j+1]
            y_label = 1 if future_c > next_o else 0
            train_X.append(X_feat)
            train_y.append(y_label)
    
    train_X = np.array(train_X)
    train_y = np.array(train_y)
    
    # 訓練データが空の場合はスキップ(安全策)
    if len(train_X) == 0:
        continue
    
    # モデル学習
    model = XGBClassifier(eval_metric='logloss')
    model.fit(train_X, train_y)
    
    # 予測
    pred = model.predict(X_current)[0]
    all_predictions.append(pred)
    all_actual.append(y_current)

# 実行時間を計算
end_time = time.time()
execution_time = end_time - start_time

# すべての予測終了後、精度評価
all_predictions = np.array(all_predictions)
all_actual = np.array(all_actual)

accuracy = accuracy_score(all_actual, all_predictions)
precision = precision_score(all_actual, all_predictions, zero_division=0)
recall = recall_score(all_actual, all_predictions, zero_division=0)
f1 = f1_score(all_actual, all_predictions, zero_division=0)

print("Execution Time:", execution_time, "seconds")
print("Accuracy:", accuracy)
print("Precision(適合率):", precision)
print("Recall(再現率):", recall)
print("F1:", f1)


Predicting: 100%|██████████| 3619/3619 [10:59<00:00,  5.49it/s]

Execution Time: 659.3745610713959 seconds
Accuracy: 0.5092567007460624
Precision(適合率): 0.5663909447104919
Recall(再現率): 0.6251802018260452
F1: 0.5943353129282778


In [9]:
!pip install statsmodels

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 11.1 MB/s eta 0:00:00a 0:00:01


In [10]:
from statsmodels.tsa.stattools import adfuller

# Return列を取得
returns = data["Return"].dropna()

# ADF検定
adf_result = adfuller(returns, autolag='AIC')

print("ADF Statistic: %f" % adf_result[0])
print("p-value: %f" % adf_result[1])
print("Critical Values:")
for key, value in adf_result[4].items():
    print('\t%s: %.3f' % (key, value))

if adf_result[1] < 0.05:
    print("有意水準5%で単位根なし（定常過程とみなせる）")
else:
    print("有意水準5%で単位根あり（非定常過程とみなせる）")

ADF Statistic: -41.790637
p-value: 0.000000
Critical Values:
	1%: -3.432
	5%: -2.862
	10%: -2.567
有意水準5%で単位根なし（定常過程とみなせる）


In [12]:
data.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Return
Date,,,,,,,,
2023-12-25 00:00:00+09:00,33414.511719,33414.511719,33221.570312,33254.031250,58900000,0.0,0.0,0.002562
2023-12-26 00:00:00+09:00,33295.679688,33312.261719,33181.359375,33305.851562,68300000,0.0,0.0,0.001558
2023-12-27 00:00:00+09:00,33532.968750,33755.750000,33521.519531,33681.238281,93700000,0.0,0.0,0.011271
2023-12-28 00:00:00+09:00,33477.468750,33571.730469,33411.238281,33539.621094,73300000,0.0,0.0,-0.004205
2023-12-29 00:00:00+09:00,33458.640625,33652.710938,33305.171875,33464.171875,89000000,0.0,0.0,-0.002250


階差

In [7]:
def create_features(data, i):
    """
    特徴量として階差を作成
    - 当日終値と過去2, 4, 8, 16営業日前終値の階差を使用
    """
    current_close = data["Close"].iloc[i]
    diff_2 = current_close - data["Close"].iloc[i-2]
    diff_4 = current_close - data["Close"].iloc[i-4]
    diff_8 = current_close - data["Close"].iloc[i-8]
    diff_16 = current_close - data["Close"].iloc[i-16]
    return np.array([diff_2, diff_4, diff_8, diff_16])

階差4o

In [13]:
# ループ設定
start_i = window_size - 1 + 16
all_predictions = []
all_actual = []

# tqdmで進行状況を表示
for i in tqdm(range(start_i, length - 16), desc="Predicting"):
    # 特徴量作成
    X_current = create_features(data, i).reshape(1, -1)
    
    # ラベル
    future_close = data["Close"].iloc[i+16]
    next_open = data["Open"].iloc[i+1]
    y_current = 1 if future_close > next_open else 0
    
    # 訓練データ作成
    train_X = []
    train_y = []
    for j in range(window_size-1, i):
        if j+16 < length:
            X_feat = create_features(data, j)
            future_c = data["Close"].iloc[j+16]
            next_o = data["Open"].iloc[j+1]
            y_label = 1 if future_c > next_o else 0
            train_X.append(X_feat)
            train_y.append(y_label)
    
    train_X = np.array(train_X)
    train_y = np.array(train_y)
    
    # 訓練データが空の場合はスキップ(安全策)
    if len(train_X) == 0:
        continue
    
    # モデル学習
    model = XGBClassifier(eval_metric='logloss')
    model.fit(train_X, train_y)
    
    # 予測
    pred = model.predict(X_current)[0]
    all_predictions.append(pred)
    all_actual.append(y_current)

# 実行時間を計算
end_time = time.time()
execution_time = end_time - start_time

# すべての予測終了後、精度評価
all_predictions = np.array(all_predictions)
all_actual = np.array(all_actual)

accuracy = accuracy_score(all_actual, all_predictions)
precision = precision_score(all_actual, all_predictions, zero_division=0)
recall = recall_score(all_actual, all_predictions, zero_division=0)
f1 = f1_score(all_actual, all_predictions, zero_division=0)

print("Execution Time:", execution_time, "seconds")
print("Accuracy:", accuracy)
print("Precision(適合率):", precision)
print("Recall(再現率):", recall)
print("F1:", f1)

Predicting: 100%|██████████| 3619/3619 [10:00<00:00,  6.03it/s]

Execution Time: 1704.0707762241364 seconds
Accuracy: 0.539651837524178
Precision(適合率): 0.5937641211025757
Recall(再現率): 0.6314271984622778
F1: 0.6120167675826735


階差o1

In [14]:
# 特徴量抽出用のウィンドウサイズ（16日間さかのぼるため）
window_size = 16

data = data_master.copy()
data = data.dropna() # 欠損値がある場合除去

length = len(data)
start_time = time.time()

start_i = window_size - 1 + 16
all_predictions = []
all_actual = []

for i in tqdm(range(start_i, length - 16), desc="Predicting"):
    # 特徴量: i日当日終値 - (i-2,i-4,i-8,i-16)日終値
    # reshape(1,-1)で2次元に
    X_current = np.array([
        data["Close"].iloc[i] - data["Close"].iloc[i-2],
        data["Close"].iloc[i] - data["Close"].iloc[i-4],
        data["Close"].iloc[i] - data["Close"].iloc[i-8],
        data["Close"].iloc[i] - data["Close"].iloc[i-16]
    ]).reshape(1, -1)
    
    # ラベル： future_close(= i+16日後Close) > next_open(= i+1日Open)なら1,そうでなければ0
    future_close = data["Close"].iloc[i+16]
    next_open = data["Open"].iloc[i+1]
    y_current = 1 if future_close > next_open else 0
    
    train_X = []
    train_y = []
    # jを16から開始することで (j-16)がマイナスにならず、常にアクセス可能
    for j in range(window_size, i):
        if j+16 < length:
            X_feat = np.array([
                data["Close"].iloc[j] - data["Close"].iloc[j-2],
                data["Close"].iloc[j] - data["Close"].iloc[j-4],
                data["Close"].iloc[j] - data["Close"].iloc[j-8],
                data["Close"].iloc[j] - data["Close"].iloc[j-16]
            ])
            
            future_c = data["Close"].iloc[j+16]
            next_o = data["Open"].iloc[j+1]
            y_label = 1 if future_c > next_o else 0
            
            train_X.append(X_feat)
            train_y.append(y_label)
    
    train_X = np.array(train_X)
    train_y = np.array(train_y)
    
    # 訓練データが空の場合はスキップ
    if len(train_X) == 0:
        continue
    
    # モデル学習
    model = XGBClassifier(eval_metric='logloss')
    model.fit(train_X, train_y)
    
    # 予測
    pred = model.predict(X_current)[0]
    all_predictions.append(pred)
    all_actual.append(y_current)

# 実行時間を計測
end_time = time.time()
execution_time = end_time - start_time

all_predictions = np.array(all_predictions)
all_actual = np.array(all_actual)

accuracy = accuracy_score(all_actual, all_predictions)
precision = precision_score(all_actual, all_predictions, zero_division=0)
recall = recall_score(all_actual, all_predictions, zero_division=0)
f1 = f1_score(all_actual, all_predictions, zero_division=0)

print("Execution Time:", execution_time, "seconds")
print("Accuracy:", accuracy)
print("Precision(適合率):", precision)
print("Recall(再現率):", recall)
print("F1:", f1)

Predicting: 100%|██████████| 3620/3620 [11:27<00:00,  5.26it/s]

Execution Time: 687.9625768661499 seconds
Accuracy: 0.5339779005524862
Precision(適合率): 0.5898135516143702
Recall(再現率): 0.6229586935638809
F1: 0.6059331931791637


一個目のコードでは j = window_size-1 のときのデータも学習用に含まれますが、二個目のコードでは j = window_size-1 が含まれず、j = window_size から始まります。つまり、一日分学習データの範囲がずれることになります。